# Shannon-Fano Coding and Galois Fields Demo

This notebook demonstrates the implementation of Shannon-Fano coding algorithm and its integration with Galois Fields for error detection and correction. We'll explore:

1. Text processing and Shannon-Fano encoding
2. Frequency analysis and optimal prefix codes
3. Galois Field construction
4. Error detection and correction
5. Binary conversions and string operations
6. Practical examples and visualizations

Let's start by importing the required libraries:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from fractions import Fraction
from typing import List, Dict, Tuple
import string
from collections import Counter

# Import our modules
from shanon_fano import string_frequencies, shanon_fano, decoder
from campos_de_galois import polynomial_form, field_constructor

ModuleNotFoundError: No module named 'pandas'

## 1. Text Processing and Shannon-Fano Encoding

The Shannon-Fano algorithm is a technique for constructing a prefix code based on a set of symbols and their probabilities (frequencies of occurrence). Let's see how it works with a simple example.

In [ ]:
# Example text
sample_text = "hello world! this is a sample text for shannon-fano coding demonstration"

# Get frequencies
data = string_frequencies(sample_text)
freqs, chars, text = data

# Create DataFrame for visualization
char_freqs = pd.DataFrame({
    'Character': chars,
    'Frequency': [float(f) for f in freqs],
    'Count': [int(float(f) * len(text)) for f in freqs]
}).sort_values('Frequency', ascending=False)

print("Character Frequencies:")
print(char_freqs)

# Plot frequencies
plt.figure(figsize=(12, 6))
plt.bar(char_freqs['Character'], char_freqs['Frequency'])
plt.title('Character Frequencies in Sample Text')
plt.xlabel('Characters')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

## 2. Shannon-Fano Tree Construction

Now let's visualize how the Shannon-Fano algorithm constructs the prefix codes by recursively dividing the symbols into groups. We'll use NetworkX to create a visual representation of the encoding tree.

In [ ]:
# Generate Shannon-Fano codes
sf_data = shanon_fano(freqs, chars)
codes = decoder(sf_data[0], sf_data[3], sf_data[2], sf_data[1], sf_data[4])

# Create a graph for visualization
G = nx.DiGraph()

# Add nodes and edges from the Shannon-Fano tree
def add_edges(graph, node_name, children):
    for i, child in enumerate(children):
        if isinstance(child, str):
            graph.add_edge(node_name, child, label=str(i))
        else:
            new_node = ''.join(str(x) for x in child)
            graph.add_edge(node_name, new_node, label=str(i))
            
for node, children in sf_data[0].items():
    if children:
        add_edges(G, node, children)

# Draw the graph
plt.figure(figsize=(15, 10))
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True, node_color='lightblue', 
        node_size=2000, font_size=10, font_weight='bold')

# Add edge labels (0/1)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels)

plt.title("Shannon-Fano Encoding Tree")
plt.show()

# Display the codes
print("\nShannon-Fano Codes:")
for char, code in codes.items():
    print(f"'{char}': {code}")

## 3. Galois Field Construction

Now let's examine how Galois Fields are constructed and used for error detection/correction. We'll create a small Galois Field GF(2³) as an example and visualize its elements.

In [ ]:
import io
import csv

# Create a small Galois Field (GF(2³))
bits = 3
output = io.StringIO()
field_constructor(bits, output)
output.seek(0)

# Read the field elements
reader = csv.reader(output)
field_data = list(reader)

# Convert to DataFrame for better visualization
columns = field_data[0]
df = pd.DataFrame(field_data[1:], columns=columns)

print("Galois Field GF(2³) Elements:")
print(df[['Index_form', 'binary', 'polynomial_form']])

# Visualize field operations
plt.figure(figsize=(12, 8))
plt.title(f'Elements of GF(2³) in Binary Form')

# Plot points in 3D space (using binary representation)
ax = plt.axes(projection='3d')
for row in df.iloc[1:].itertuples():
    binary = row.binary.replace('₂', '')
    coords = [int(b) for b in binary]
    ax.scatter3D(coords[0], coords[1], coords[2])
    ax.text(coords[0], coords[1], coords[2], row.Index_form)

ax.set_xlabel('b₂')
ax.set_ylabel('b₁')
ax.set_zlabel('b₀')
plt.show()

## 4. Error Detection and Correction

Now let's demonstrate how we can use Galois Fields for error detection and correction. We'll encode a message, introduce some errors, and then detect and correct them.

In [ ]:
from campos_de_galois import string_to_binary, binary_to_string, binary_errors

# Original message
message = "Hello"
print(f"Original message: {message}")

# Convert to binary
binary = string_to_binary(message)
print(f"Binary representation: {binary}")

# Introduce errors
corrupted = binary_errors(binary)
print(f"Corrupted binary: {corrupted}")

# Try to decode corrupted message
decoded = binary_to_string(corrupted)
print(f"Decoded (with errors): {decoded}")

# Visualize the differences
def plot_binary_comparison(original, corrupted):
    plt.figure(figsize=(15, 5))
    
    # Split into individual bits
    orig_bits = ''.join(original.split())
    corr_bits = ''.join(corrupted.split())
    
    # Plot original
    plt.subplot(2, 1, 1)
    plt.plot([int(b) for b in orig_bits], 'b-', label='Original')
    plt.title('Original vs Corrupted Binary Data')
    plt.ylabel('Bit Value')
    plt.legend()
    plt.grid(True)
    
    # Plot corrupted
    plt.subplot(2, 1, 2)
    plt.plot([int(b) for b in corr_bits], 'r-', label='Corrupted')
    plt.xlabel('Bit Position')
    plt.ylabel('Bit Value')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()

plot_binary_comparison(binary, corrupted)

## 5. Complete Example: Text Processing Pipeline

Let's put everything together in a complete example that shows the entire process from text input to encoded data with error protection.

In [ ]:
def process_text(text):
    # 1. Get character frequencies
    freqs, chars, _ = string_frequencies(text)
    
    # 2. Generate Shannon-Fano codes
    sf_data = shanon_fano(freqs, chars)
    codes = decoder(sf_data[0], sf_data[3], sf_data[2], sf_data[1], sf_data[4])
    
    # 3. Convert to binary
    binary = string_to_binary(text)
    
    # 4. Calculate required Galois Field size
    max_code_length = max(len(''.join(map(str, code))) for code in codes.values())
    field_size = 2 ** max_code_length
    
    return {
        'frequencies': pd.DataFrame({'char': chars, 'frequency': [float(f) for f in freqs]}),
        'codes': codes,
        'binary': binary,
        'field_size': field_size
    }

# Process a sample text
sample = "The quick brown fox jumps over the lazy dog"
results = process_text(sample)

# Display results
print("Input text:", sample)
print("\nCharacter frequencies:")
print(results['frequencies'])
print("\nShannon-Fano codes:")
for char, code in results['codes'].items():
    print(f"'{char}': {code}")
print("\nBinary representation:", results['binary'])
print("\nRequired Galois Field: GF({})\n".format(results['field_size']))

# Visualize code lengths
code_lengths = {k: len(''.join(map(str, v))) for k, v in results['codes'].items()}
plt.figure(figsize=(12, 6))
plt.bar(code_lengths.keys(), code_lengths.values())
plt.title('Code Lengths for Each Character')
plt.xlabel('Character')
plt.ylabel('Code Length (bits)')
plt.xticks(rotation=45)
plt.show()

## Conclusion

This notebook has demonstrated the practical application of Shannon-Fano coding and Galois Fields for text encoding and error correction. Key takeaways:

1. Shannon-Fano coding provides an efficient way to encode text based on character frequencies
2. Galois Fields provide a mathematical foundation for error detection and correction
3. The combination of these techniques enables reliable data transmission

For more information, see:
- Shannon's Information Theory
- Finite Field Theory
- Error Correction Codes